In [1]:
import pandas as pd
import spacy
import tbdb
import matplotlib.pyplot as plt
import seaborn as sns
from openai import OpenAI

In [ ]:
def find_emotion_word(utterance, emotion_words):
    # extract emotion words from the utterance according to our lists. 
    emotions_found = []
    utterance = utterance.split()
    for emotion in emotion_words:
        if emotion in utterance:
            emotions_found.append(emotion)
    return emotions_found

In [ ]:

#import OpenAI
client = ... # enter the keys here
def label_row(row, word):
    messages = [ {"role": "system", "content":  
                  "You are a intelligent assistant."} ] 
    message = f"I am identifying what context the word {word} is being used by children. Please provide a one word label (don't give me a sentence) for the context in which {word} is used by the CHI (Child) in the last sentence. Emotion would be the label if it is being used in an emotion context. If not what is the context in one word?:"+str(row)
    if message: 
        messages.append( 
            {"role": "user", "content": message}, 
        ) 
        chat = client.chat.completions.create( 
            model="gpt-3.5-turbo", messages=messages 
        ) 
    reply = chat.choices[0].message.content 
    #print(f"ChatGPT: {reply}") 
    return reply

In [2]:
all_utt_data = pd.read_csv('eng-NA-all-utterances.csv')
all_parts_data = pd.read_csv('eng-NA-all-participants.csv')

C:\Users\luliu\AppData\Local\Temp\ipykernel_37312\2327413080.py:1: DtypeWarning: Columns (0,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  all_utt_data = pd.read_csv('eng-NA-all-utterances.csv')


In [3]:
# emotion word lists
final_emotion_list = ['happy', 'sad', 'afraid', 'angry', 'scared', 'surprised', 'excited', 'upset', 'glad', 'frightened', \
'unhappy', 'bored', 'lonely', 'annoyed', 'disappointed', 'ashamed', 'pleased', 'worried', 'calm', 'embarrassed', 'lonesome', \
'disgusted', 'cheerful', 'jealous', 'furious', 'delighted', 'unafraid', 'fed up', 'concerned', 'miserable', 'frustrated', \
'anxious', 'fearful', 'troubled', 'joyful', 'depressed', 'gloomy', 'merry', 'terrified', 'joyous', 'shocked', 'hopeful', \
'relieved', 'content', 'hopeless', 'carefree', 'envious', 'at ease', 'tense', 'mad', 'insecure', 'good']

In [5]:
all_emotion_data = all_utt_data
all_emotion_data["emotion_words"] = all_emotion_data['utterance'].astype(str).apply(find_emotion_word, emotion_words=final_emotion_list)
all_emotion_data = all_emotion_data[all_emotion_data['emotion_words'].apply(len) > 0]

In [6]:
# extract the child participants
child_part = all_parts_data[all_parts_data["who"] == "CHI"]

In [7]:
# delete rows without emotion words
Laura = all_emotion_data[(all_emotion_data['path'].str.contains('Braunwald')) & (all_utt_data['who']=='CHI')]
Laura = Laura[Laura['emotion_words'].apply(len) > 0]

C:\Users\luliu\AppData\Local\Temp\ipykernel_37312\1117228299.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  Laura = all_emotion_data[(all_emotion_data['path'].str.contains('Braunwald')) & (all_utt_data['who']=='CHI')]


In [8]:
# extract context for child Laura
i = 0
Laura['context'] = [[] for _ in range(len(Laura))]
for index, row in Laura.iterrows():
    i += 1
    print(i)
    # Find the subset of 'all_utt_data' DataFrame where the 'path' matches the current row's 'path'
    matched_df = all_utt_data[all_utt_data['path'] == row['path']]
    # Find the index of the row in 'matched_df' where 'utt_num' matches the current row's 'utt_num'
    match_index = matched_df.index[matched_df['utt_num'] == row['utt_num']].tolist()
    if match_index[0] > 0:
        # Determine the starting index for context, 5 rows before the match_index
        start_index = match_index[0] - 5
        while start_index < 0:
        # one row after 
            start_index += 1
        context_rows = all_utt_data.iloc[start_index:match_index[0]+1]    
    context_list = [f"{row['who']}:{row['utterance']}" for _, row in context_rows.iterrows()]
    # Assign the 'context_list' to the 'context' column in 'Laura' for the current index
    Laura.at[index, 'context'] = context_list


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [10]:
child_part.drop_duplicates(inplace=True)

C:\Users\luliu\AppData\Local\Temp\ipykernel_6360\2960699825.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  child_part.drop_duplicates(inplace=True)


In [9]:
Laura = pd.merge(Laura, child_part[['path', 'age']], on=['path'], how='left')

In [10]:
# create age groups
Laura['age'] = Laura['age'].str.replace(r'\..*', '', regex=True)
Laura[['years', 'months']] = Laura['age'].str.split(';', expand=True)
Laura['years'] = pd.to_numeric(Laura['years'], errors='coerce')
Laura['months'] = pd.to_numeric(Laura['months'], errors='coerce')
Laura['months_decimal'] = round(Laura['months'] / 12, 2)
Laura['decimal_age'] = Laura['years'] + Laura['months_decimal']
Laura.drop(['years', 'months', 'months_decimal', 'age'], axis=1, inplace=True)
Laura['age_group'] = Laura['decimal_age'].apply(lambda x: f"{int(x)}-{int(x)+1}")

In [11]:
# look at utterances with emotion word good
Laura_good = Laura[Laura['emotion_words'].apply(lambda x: 'good' in x)].reset_index()

In [15]:
### label context in batch
# Number of rows per batch
batch_size = 10

# Create batches and apply the labeling function
batches = [Laura_good.iloc[i:i+batch_size] for i in range(0, len(Laura_good), batch_size)]
labels = [batch['context'].apply(label_row, word='good') for batch in batches]

# Flatten the list of lists
labels_flat = [label for sublist in labels for label in sublist]

# Assign labels back to the DataFrame
Laura_good['label'] = labels_flat


In [17]:
Laura_good.to_csv('Laura_good.csv')

In [29]:
Laura_good = pd.read_csv('Laura_good.csv')

In [41]:
Laura_good

,Unnamed: 0,index,filename,path,utt_num,who,role,postcodes,gems,utterance,startTime,endTime,emotion_words,context,decimal_age,age_group,label
0,0,4,010615,childes/Eng-NA/Braunwald/010615,227,CHI,Target_Child,I,NaN,oh good,710.506,712.143,['good'],"[""MOT:you're bypassing the bowl in front of me...",1.50,1-2,emotion
1,1,9,010621,childes/Eng-NA/Braunwald/010621,108,CHI,Target_Child,NaN,NaN,good,223.494,224.028,['good'],"[""MOT:that's right"", 'CHI:Swww', 'CHI:Swww', '...",1.50,1-2,approval
2,2,11,010707,childes/Eng-NA/Braunwald/010707,259,CHI,Target_Child,NaN,NaN,good doggy,489.898,491.721,['good'],"['MOT:maybe he is', ""SIS:Gwww isn't a kitty"", ...",1.58,1-2,praise
3,3,12,010707,childes/Eng-NA/Braunwald/010707,261,CHI,Target_Child,SR,NaN,good doggy,492.382,494.136,['good'],"['CHI:he more than you', ""SIS:you know Gwww is...",1.58,1-2,emotion
4,4,13,010707,childes/Eng-NA/Braunwald/010707,262,CHI,Target_Child,SR,NaN,good doggy,494.321,496.248,['good'],"[""SIS:you know Gwww isn't a kitty"", 'CHI:yeah'...",1.58,1-2,affection
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,194,374,030301,childes/Eng-NA/Braunwald/0diary/030301,21,CHI,Target_Child,NaN,NaN,good,NaN,NaN,['good'],"['MOT:I saved the house jobs for you', 'CHI:oh...",3.25,3-4,approval
195,195,376,030309,childes/Eng-NA/Braunwald/0diary/030309,1,CHI,Target_Child,NaN,NaN,that's a good idea,NaN,NaN,['good'],"['CHI:more cow cheese', 'MOT:okay here you go'...",3.25,3-4,approval
196,196,388,030406,childes/Eng-NA/Braunwald/0diary/030406,5,CHI,Target_Child,NaN,NaN,could I lick this since I did a good job,NaN,NaN,['good'],"[""CHI:Daddy I'm getting better at my letters"",...",3.33,3-4,task
197,197,394,030417,childes/Eng-NA/Braunwald/0diary/030417,10,CHI,Target_Child,NaN,NaN,milk is good better than water Joanna,NaN,NaN,['good'],"[""CHI:I get the rest since I'm the nut server""...",3.33,3-4,preference


In [33]:
label_lst = list(Laura_good['label'].unique())

In [36]:
# remove any potential punctuation generated by openai reponse
import string
for i in range(len(label_lst)):
    label_lst[i] = label_lst[i].lower().strip(string.punctuation)

In [38]:
label_lst = list(set(label_lst))
label_lst

['care',
 'happiness',
 'unclear',
 'emotion',
 'greeting',
 'judgment',
 'taste',
 'desire',
 'skill',
 'action',
 'family',
 'behavior',
 'capability',
 'desirability',
 'health',
 'encouragement',
 'repetition',
 'preference',
 'appreciation',
 'evaluation',
 'animals',
 'wishful',
 'ability',
 'food',
 'affection',
 'possession',
 'compliance',
 'interaction',
 'response',
 'approval',
 'manners',
 'sound',
 'character',
 'beneficial',
 'wish',
 'enjoyment',
 'relationship',
 'task',
 'comparison',
 'achievement',
 'opinion',
 'positive',
 'approbation',
 'certainty',
 'praise',
 'accomplishment',
 'quality',
 'satisfaction']

In [40]:
# remove any punctuation
Laura_good['label'] = Laura_good['label'].str.lower().str.rstrip('.!?')

In [42]:
# extract one example for each unique label
first_rows = []
for label in label_lst:
    match = Laura_good[Laura_good['label'].str.lower().str.rstrip('.!?') == label].head(1)
    if not match.empty:
        first_rows.append(match)
laura_good_unique = pd.concat(first_rows)

In [46]:
laura_good_unique.to_csv('laura_good_unique.csv')